# Import Libraries

In [38]:
import sys
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem
from statistics import mean
from rdkit.Chem import Descriptors, rdMolDescriptors, PandasTools, AllChem
from scipy.stats import loguniform, randint, uniform
from boruta import BorutaPy
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score, confusion_matrix,matthews_corrcoef
from sklearn.metrics import make_scorer, classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, f1_score, roc_curve, precision_score, cohen_kappa_score, accuracy_score, r2_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, KFold
from sklearn.model_selection import cross_validate
from sklearn.utils import resample
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

# Load your descriptor file here

In [41]:
data = pd.read_csv(r"C:\Users\wanih\CXCR4\Dataset_external_descriptors.csv")

In [42]:
data

,Unnamed: 0,Code,Smiles,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,0,IS00002,CCOC(=O)C1=C(C)NC(=O)NC1c1ccc(cc1)N(C)C,12.194467,12.194467,0.287547,-0.510174,0.834057,17.636364,303.362,...,0,0,0,0,0,0,0,0,0,1
1,1,IS00003,CC1Oc2cc(O)ccc2C(=O)C1C,11.776612,11.776612,0.082176,-0.127963,0.683360,25.428571,192.214,...,0,0,0,0,0,0,0,0,0,0
2,2,IS00004,COc1ccc2C(=O)CC(C)(C)Oc2c1,11.779947,11.779947,0.128614,-0.420190,0.707556,17.933333,206.241,...,0,0,0,0,0,0,0,0,0,0
3,3,IS00005,Oc1ccc2C(=O)CC3(CCCC3)Oc2c1,11.996357,11.996357,0.141806,-0.279884,0.727797,21.875000,218.252,...,0,0,0,0,0,0,0,0,0,0
4,4,IS00006,Oc1ccc2C(=O)CC3(CCCCC3)Oc2c1,12.108602,12.108602,0.146435,-0.295509,0.747378,22.000000,232.279,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2141,2141,IS002500,COc1ccc(cc1)C(=O)N1CCCCC1,12.084461,12.084461,0.141068,0.141068,0.763612,15.937500,219.284,...,0,0,0,0,0,0,0,0,0,0
2142,2142,IS002501,O=C(N1CCCCC1)c1ccccc1,11.925648,11.925648,0.185463,0.185463,0.663533,16.714286,189.258,...,0,0,0,0,0,0,0,0,0,0
2143,2143,IS002502,Cc1ccc(N2CCCC2)c(C)c1,2.494815,2.494815,1.239981,1.239981,0.634054,16.615385,175.275,...,0,0,0,0,0,0,0,0,0,0
2144,2144,IS002503,O=C(CCc1ccccc1)C#Cc1ccccc1,11.616080,11.616080,0.011383,-0.011383,0.745040,9.333333,234.298,...,0,0,0,0,0,0,0,0,0,0


In [43]:
eX_data =data.iloc[:, 3:]

In [44]:
eX_data

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,12.194467,12.194467,0.287547,-0.510174,0.834057,17.636364,303.362,282.194,303.158292,118,...,0,0,0,0,0,0,0,0,0,1
1,11.776612,11.776612,0.082176,-0.127963,0.683360,25.428571,192.214,180.118,192.078644,74,...,0,0,0,0,0,0,0,0,0,0
2,11.779947,11.779947,0.128614,-0.420190,0.707556,17.933333,206.241,192.129,206.094294,80,...,0,0,0,0,0,0,0,0,0,0
3,11.996357,11.996357,0.141806,-0.279884,0.727797,21.875000,218.252,204.140,218.094294,84,...,0,0,0,0,0,0,0,0,0,0
4,12.108602,12.108602,0.146435,-0.295509,0.747378,22.000000,232.279,216.151,232.109944,90,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2141,12.084461,12.084461,0.141068,0.141068,0.763612,15.937500,219.284,202.148,219.125929,86,...,0,0,0,0,0,0,0,0,0,0
2142,11.925648,11.925648,0.185463,0.185463,0.663533,16.714286,189.258,174.138,189.115364,74,...,0,0,0,0,0,0,0,0,0,0
2143,2.494815,2.494815,1.239981,1.239981,0.634054,16.615385,175.275,158.139,175.136100,70,...,0,0,0,0,0,0,0,0,0,0
2144,11.616080,11.616080,0.011383,-0.011383,0.745040,9.333333,234.298,220.186,234.104465,88,...,0,0,0,0,0,0,0,0,0,0


# Normilization of data

In [45]:
with open("standard_scaler.pkl", "rb") as f:
    scaler = pickle.load(f)
data_scaled = scaler.transform(eX_data)

In [46]:
list2=eX_data.columns
external1= pd.DataFrame(data_scaled,columns=list2)

# Selected fatures

In [47]:
with open("selected_features.pkl", "rb") as f:
    selected_features = pickle.load(f)
# Apply to new data
selected = external1[selected_features]

In [48]:
selected

,BCUT2D_MWHI,BCUT2D_MWLOW,BCUT2D_CHGHI,BCUT2D_CHGLO,BCUT2D_LOGPHI,BCUT2D_LOGPLOW,BCUT2D_MRHI,BCUT2D_MRLOW,PEOE_VSA5,PEOE_VSA7,PEOE_VSA8,SMR_VSA10,SMR_VSA3,EState_VSA4,VSA_EState10,VSA_EState2,VSA_EState7,fr_Ar_N
0,-0.355215,0.186292,0.595139,0.588798,-0.362049,-0.510570,-0.045436,-1.575467,-0.372574,-2.218563,-0.702629,0.121380,-0.995497,-1.976421,-0.39884,1.152488,-1.895091,-1.306181
1,-0.360626,-0.257443,0.482197,0.685488,0.185350,0.662498,0.032757,-0.216065,-0.372574,-2.998969,-1.713234,-0.613361,-2.091318,-2.522276,-0.39884,-0.429468,-1.927200,-1.306181
2,-0.358699,-0.075248,0.649594,0.201347,0.465786,0.423731,0.019717,-0.287416,-0.372574,-2.566259,-1.713234,-0.613361,-2.091318,-2.522276,-0.39884,-0.429095,-1.788962,-1.306181
3,-0.358728,-0.615405,1.833283,-0.175421,1.552282,0.088381,0.021184,-0.396350,-0.372574,-1.826563,-1.713234,-0.613361,-2.091318,-1.276205,-0.39884,-0.404890,-0.728751,-1.306181
4,-0.358764,-1.847865,1.674980,-0.918825,1.417039,-0.330327,0.020957,-0.598333,-0.372574,-1.826563,-1.713234,-0.613361,-2.091318,-1.276205,-0.39884,-0.392335,-0.420520,-1.306181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2141,-0.364285,1.902533,-1.070882,0.799641,-1.988527,0.453111,-0.053016,-0.219616,-0.372574,-1.469588,-1.058012,-0.605712,-1.586368,-0.715310,-0.39884,-0.178517,-1.017796,-1.306181
2142,-0.406365,1.938398,-1.134948,0.811023,-2.449159,0.454960,-0.055489,-0.219591,-0.372574,-2.227842,-1.058012,-0.605712,-1.586368,-0.994258,-0.39884,-0.193892,-1.000487,-1.306181
2143,-0.542598,2.076592,-1.564163,1.609782,-0.909422,2.116043,-0.527085,5.415745,-0.372574,-1.795132,-1.051561,-0.619278,-2.091318,-2.522276,-0.39884,-1.467620,-1.215339,-1.306181
2144,-0.408449,1.740703,-3.329982,4.545275,-3.700040,4.257154,-0.032800,-1.413036,-0.372574,-2.312697,-1.405165,-0.613361,-2.091318,-1.940837,-0.39884,-0.447423,-1.586662,-1.306181


In [49]:
import pandas as pd
import pickle

# Load saved models
with open("dt_model.pkl", "rb") as f:
    mod_dt = pickle.load(f)

with open("lr_model.pkl", "rb") as f:
    mod_lr = pickle.load(f)

with open("adbt_model.pkl", "rb") as f:
    mod_adbt = pickle.load(f)

# External feature set
df_predict_features = selected

# Predictions
df_predict_clf = pd.DataFrame()
df_predict_clf['Class_pred_DecisionTree'] = mod_dt.predict(df_predict_features)
df_predict_clf['Class_pred_LogisticRegression'] = mod_lr.predict(df_predict_features)
df_predict_clf['Class_pred_AdaBoost'] = mod_adbt.predict(df_predict_features)

# Metadata
t1 = pd.DataFrame(data['Code'].values, columns=['Code'])
t2 = pd.DataFrame(data['Smiles'].values, columns=['Smiles'])

# Final output
result = pd.concat([t1, t2, df_predict_clf], axis=1)

result 

,Code,Smiles,Class_pred_DecisionTree,Class_pred_LogisticRegression,Class_pred_AdaBoost
0,IS00002,CCOC(=O)C1=C(C)NC(=O)NC1c1ccc(cc1)N(C)C,0,0,0
1,IS00003,CC1Oc2cc(O)ccc2C(=O)C1C,0,0,0
2,IS00004,COc1ccc2C(=O)CC(C)(C)Oc2c1,0,0,0
3,IS00005,Oc1ccc2C(=O)CC3(CCCC3)Oc2c1,1,1,0
4,IS00006,Oc1ccc2C(=O)CC3(CCCCC3)Oc2c1,1,0,0
...,...,...,...,...,...
2141,IS002500,COc1ccc(cc1)C(=O)N1CCCCC1,1,0,0
2142,IS002501,O=C(N1CCCCC1)c1ccccc1,1,0,0
2143,IS002502,Cc1ccc(N2CCCC2)c(C)c1,1,1,0
2144,IS002503,O=C(CCc1ccccc1)C#Cc1ccccc1,0,0,0
